Calculate eddy heat flux. 

In [1]:
import numpy as np
import math
import xarray as xr 
import pickle 
import matplotlib.pyplot as plt

/knight/anaconda_jan21/envs/aug21/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
infile = open("1959gph100.p", 'rb') 
v100 = pickle.load(infile)
infile.close()

infile = open("1959temp100.p", 'rb') 
t100 = pickle.load(infile)
infile.close()

In [3]:
infile = open("era5_lonindex.p", 'rb') 
lon = pickle.load(infile)
infile.close()

infile = open("era5_latindex.p", 'rb') 
lat = pickle.load(infile)
infile.close()

infile = open("era5_timeindex.p", 'rb') 
time = pickle.load(infile)
infile.close()

In [4]:
##save year index
year = [i for i in range(1959,2022)] #indicate years for data, needed for opening files

In [5]:
v100.shape

(64, 152, 81, 1440)

In [6]:
t100.shape

(63, 152, 81, 1440)

In [8]:
t100 = xr.DataArray(data= t100, dims = ["year","day","lat","lon"],coords = dict(year = year,
                                                                                day = time,
                                                                                lat = lat[120:201],
                                                                                lon = lon))

In [10]:
year = [i for i in range(1959,2023)] #indicate years for data, needed for opening files

In [11]:
v100 = xr.DataArray(data= v100, dims = ["year","day","lat","lon"],coords = dict(year = year,
                                                                                day = time,
                                                                                lat = lat[120:201],
                                                                                lon = lon))

In [13]:
v100

<xarray.DataArray (year: 64, day: 152, lat: 81, lon: 1440)>
array([[[[15830.36760958, 15830.36760958, 15829.6110474 , ...,
          15830.36760958, 15830.36760958, 15830.36760958],
         [15835.66832314, 15835.66832314, 15835.66832314, ...,
          15835.66832314, 15835.66832314, 15835.66832314],
         [15840.97062946, 15840.97062946, 15840.97062946, ...,
          15840.97062946, 15840.97062946, 15840.97062946],
         ...,
         [16324.13035168, 16321.85907238, 16320.34435525, ...,
          16329.43106524, 16327.91794088, 16325.64666157],
         [16329.43106524, 16327.15978593, 16324.88850663, ...,
          16334.73337156, 16333.21865443, 16330.94737513],
         [16334.73337156, 16332.46049949, 16330.18922018, ...,
          16340.03408512, 16338.51936799, 16336.24808869]],

        [[15793.25942915, 15793.25942915, 15793.25942915, ...,
          15793.25942915, 15793.25942915, 15793.25942915],
         [15795.53230122, 15795.53230122, 15795.53230122, ...,
          15795.53230122, 15795.53230122, 15795.53230122],
         [15797.80517329, 15797.80517329, 15797.80517329, ...,
          15797.80517329, 15797.80517329, 15797.80517329],
...
         [16272.80517329, 16271.31116208, 16270.56256371, ...,
          16274.30237003, 16274.30237003, 16273.55377166],
         [16275.79956677, 16275.0509684 , 16273.55377166, ...,
          16278.04217635, 16277.29517074, 16276.54657238],
         [16278.79077472, 16278.04217635, 16276.54657238, ...,
          16281.78357543, 16281.03497706, 16280.28797146]],

        [[15956.36308614, 15957.11168451, 15958.60728848, ...,
          15953.37028542, 15954.11888379, 15954.86748216],
         [15958.60728848, 15959.35588685, 15960.85149083, ...,
          15955.61608053, 15956.36308614, 15957.85869011],
         [15960.85149083, 15961.60008919, 15963.09569317, ...,
          15957.85869011, 15958.60728848, 15960.10448522],
         ...,
         [16248.1189475 , 16247.3719419 , 16246.62334353, ...,
          16249.61455148, 16248.86754587, 16248.86754587],
         [16254.10295617, 16253.35595056, 16251.86034659, ...,
          16256.34875127, 16255.60015291, 16254.85155454],
         [16260.08855759, 16258.59136086, 16257.84435525, ...,
          16262.33275994, 16261.58575433, 16260.83715596]]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2018 2019 2020 2021 2022
  * day      (day) <U5 '11-01' '11-02' '11-03' ... '03-29' '03-30' '03-31'
  * lat      (lat) float32 60.0 59.75 59.5 59.25 59.0 ... 40.75 40.5 40.25 40.0
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8

In [18]:
#select area
t = t100.loc[dict(lat=slice(60,40))]
v = v100.loc[dict(lat=slice(60,40),year=slice(1959,2021))]

In [19]:
#create weights
t_weights=np.cos(np.deg2rad(t.lat))
zm_t = t.weighted(t_weights).mean(dim="lon")#average over the full area
zm_t ##zonal mean temp

<xarray.DataArray (year: 63, day: 152, lat: 81)>
array([[[219.18703378, 219.13331593, 219.08044784, ..., 210.5144629 ,
         210.35179376, 210.20540756],
        [218.59888159, 218.58117783, 218.56821067, ..., 210.8131795 ,
         210.68620243, 210.545191  ],
        [217.89016691, 217.9028291 , 217.90295403, ..., 210.79887367,
         210.6199886 , 210.43480456],
        ...,
        [220.51055414, 220.54801667, 220.58451901, ..., 214.8886983 ,
         214.69537648, 214.50621106],
        [220.13057821, 220.17693535, 220.23027275, ..., 215.40441754,
         215.20367055, 215.01597912],
        [220.00349675, 220.0521486 , 220.10168929, ..., 215.37884501,
         215.16407294, 214.95369515]],

       [[218.88687799, 218.85566914, 218.81824182, ..., 210.13040899,
         209.96719175, 209.81355502],
        [218.76457738, 218.73382788, 218.69376365, ..., 210.45658394,
         210.25679868, 210.06716508],
        [218.21760009, 218.19879392, 218.17219685, ..., 210.55850595,
         210.39274138, 210.24238326],
...
        [217.76838721, 217.75600448, 217.73888677, ..., 213.340568  ,
         213.14174492, 212.947288  ],
        [217.58047146, 217.55819644, 217.53477701, ..., 213.00478787,
         212.81895694, 212.63629493],
        [216.817547  , 216.77891444, 216.74139562, ..., 210.8317387 ,
         210.63977106, 210.46323195]],

       [[216.817547  , 216.77891444, 216.74139562, ..., 210.8317387 ,
         210.63977106, 210.46323195],
        [216.69079858, 216.69644774, 216.7020932 , ..., 211.40043132,
         211.23920102, 211.07249151],
        [216.83485315, 216.85256289, 216.86998656, ..., 211.74137974,
         211.53495388, 211.32564895],
        ...,
        [218.3832699 , 218.3699088 , 218.35723016, ..., 213.31646666,
         213.12844527, 212.93527174],
        [218.29187986, 218.24538419, 218.20226841, ..., 213.27326269,
         213.09722375, 212.91946392],
        [218.16549124, 218.17851191, 218.18643193, ..., 210.41513412,
         210.25234667, 210.08664752]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) <U5 '11-01' '11-02' '11-03' ... '03-29' '03-30' '03-31'
  * lat      (lat) float32 60.0 59.75 59.5 59.25 59.0 ... 40.75 40.5 40.25 40.0

In [20]:
#create weights
v_weights=np.cos(np.deg2rad(v.lat))
zm_v = v.weighted(v_weights).mean(dim="lon")#average over the full area
zm_v ##zonal mean meridional

<xarray.DataArray (year: 63, day: 152, lat: 81)>
array([[[15848.907401  , 15853.36845817, 15857.99358879, ...,
         16258.06556734, 16262.53637215, 16267.46768612],
        [15828.71366305, 15832.71005814, 15836.87829688, ...,
         16245.94107891, 16250.65780606, 16255.85570052],
        [15819.9041997 , 15823.67143988, 15827.607788  , ...,
         16237.40438769, 16242.51154769, 16248.09913974],
        ...,
        [15713.84796986, 15718.22797868, 15722.77458728, ...,
         16104.48769518, 16109.35431625, 16114.72751546],
        [15732.47425914, 15736.68859976, 15741.08901299, ...,
         16112.82472175, 16117.70210984, 16123.05920117],
        [15745.20837674, 15749.05290092, 15753.02392604, ...,
         16113.04464367, 16117.97754879, 16123.44244046]],

       [[15826.61044558, 15831.03419821, 15835.69804674, ...,
         16267.23029007, 16272.16478771, 16277.55248392],
        [15827.78125545, 15832.01228026, 15836.39593806, ...,
         16259.09902392, 16264.11435997, 16269.63920432],
        [15827.19222089, 15831.07186551, 15835.13618277, ...,
         16243.18836777, 16248.11564198, 16253.52626118],
...
        [15806.26612189, 15812.80048676, 15819.45652085, ...,
         16219.72779591, 16224.20129624, 16229.18456892],
        [15831.55783919, 15837.73718442, 15843.9454222 , ...,
         16225.17940161, 16229.22773004, 16233.76078109],
        [15857.74464036, 15862.93983163, 15868.31155904, ...,
         16286.84687022, 16291.85488188, 16297.34381982]],

       [[15857.74464036, 15862.93983163, 15868.31155904, ...,
         16286.84687022, 16291.85488188, 16297.34381982],
        [15864.47224364, 15868.37916341, 15872.43015056, ...,
         16259.44474459, 16264.65335638, 16270.34099246],
        [15875.74302836, 15878.39677733, 15881.22770219, ...,
         16234.91553529, 16240.18829975, 16245.97026771],
        ...,
        [15741.65538699, 15744.93623253, 15748.41151906, ...,
         16198.9172762 , 16204.89216422, 16211.27894134],
        [15765.17709052, 15768.4113397 , 15771.84692878, ...,
         16192.20910863, 16198.00727099, 16204.27842323],
        [15835.31653422, 15839.75802497, 15844.35437251, ...,
         16277.83575823, 16283.0075479 , 16288.63741278]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) <U5 '11-01' '11-02' '11-03' ... '03-29' '03-30' '03-31'
  * lat      (lat) float32 60.0 59.75 59.5 59.25 59.0 ... 40.75 40.5 40.25 40.0

In [21]:
ehf = ((v - zm_v) * (t - zm_t))

In [22]:
ehf

<xarray.DataArray (year: 63, day: 152, lat: 81, lon: 1440)>
array([[[[-4.69686091e+01, -4.62806091e+01, -4.74136737e+01, ...,
          -4.88028988e+01, -4.82297540e+01, -4.76566092e+01],
         [-4.61571229e+01, -4.57193190e+01, -4.52083226e+01, ...,
          -4.72150365e+01, -4.68866161e+01, -4.65584657e+01],
         [-4.42033729e+01, -4.40981743e+01, -4.39579094e+01, ...,
          -4.42384391e+01, -4.42735053e+01, -4.42384391e+01],
         ...,
         [-6.59849862e+01, -5.67468230e+01, -4.65416868e+01, ...,
          -8.29005830e+01, -7.91273402e+01, -7.29318560e+01],
         [-8.28203130e+01, -7.45464406e+01, -6.44720422e+01, ...,
          -9.60818016e+01, -9.37747713e+01, -8.86455236e+01],
         [-9.74196620e+01, -9.01084957e+01, -8.07539657e+01, ...,
          -1.06443035e+02, -1.05538451e+02, -1.01739698e+02]],

        [[-2.07357550e+02, -2.06919349e+02, -2.06261506e+02, ...,
          -2.07284516e+02, -2.07430583e+02, -2.07503616e+02],
         [-2.24609524e+02, -2.24073437e+02, -2.23077280e+02, ...,
          -2.24609524e+02, -2.24839842e+02, -2.24839842e+02],
         [-2.42527351e+02, -2.41399921e+02, -2.39788965e+02, ...,
          -2.43735270e+02, -2.43654782e+02, -2.43252341e+02],
...
         [ 1.10300671e+02,  1.08535307e+02,  1.09873395e+02, ...,
           1.10417724e+02,  1.09094938e+02,  1.08030330e+02],
         [ 1.12438924e+02,  1.10008308e+02,  1.10595312e+02, ...,
           1.14542740e+02,  1.12630008e+02,  1.10942879e+02],
         [ 1.14040098e+02,  1.10731919e+02,  1.10329781e+02, ...,
           1.17819459e+02,  1.15527650e+02,  1.13256667e+02]],

        [[-9.51251785e+01, -8.78619730e+01, -8.11277372e+01, ...,
          -1.12615622e+02, -1.07636295e+02, -1.01644793e+02],
         [-8.64516910e+01, -7.97612509e+01, -7.20461148e+01, ...,
          -1.06839431e+02, -1.00938527e+02, -9.40272676e+01],
         [-8.01041245e+01, -7.29507230e+01, -6.76695200e+01, ...,
          -1.01952352e+02, -9.51151902e+01, -8.76566583e+01],
         ...,
         [-7.92216851e+02, -7.94391239e+02, -7.95151407e+02, ...,
          -7.20519520e+02, -7.53903359e+02, -7.77572902e+02],
         [-7.87938423e+02, -7.87615936e+02, -7.80691938e+02, ...,
          -7.47723612e+02, -7.74739013e+02, -7.86095792e+02],
         [-7.78922370e+02, -7.70004606e+02, -7.62791431e+02, ...,
          -7.65232594e+02, -7.79145511e+02, -7.80906858e+02]]]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) <U5 '11-01' '11-02' '11-03' ... '03-29' '03-30' '03-31'
  * lat      (lat) float32 60.0 59.75 59.5 59.25 59.0 ... 40.75 40.5 40.25 40.0
  * lon      (lon) float32 -180.0 -179.8 -179.5 -179.2 ... 179.2 179.5 179.8

In [23]:
ehf_weights=np.cos(np.deg2rad(ehf.lat))
zm_ehf = ehf.weighted(ehf_weights).mean(dim="lon").mean(dim='lat')#average over the full area
zm_ehf ##zonal mean meridional eddy heat flux

<xarray.DataArray (year: 63, day: 152)>
array([[-404.20783479, -450.92024203, -373.67414475, ...,  235.6661202 ,
         167.54786825,  110.67723254],
       [-229.92692639, -273.30038605, -291.00705468, ...,   19.70282958,
         -76.39184291, -379.97826204],
       [-379.97826204, -303.58959387, -323.31485488, ..., -232.21986642,
        -238.70337818, -239.65967076],
       ...,
       [-140.86985504, -191.5757589 , -142.89460546, ...,  -19.1009854 ,
         -84.67226611,  -76.87184308],
       [-312.93831433, -438.34425399, -511.10772938, ...,   68.40310393,
          89.40571451, -172.61364831],
       [-172.61364831, -185.77374608, -269.99928109, ..., 1207.81408812,
        1132.33381183, -223.76357151]])
Coordinates:
  * year     (year) int64 1959 1960 1961 1962 1963 ... 2017 2018 2019 2020 2021
  * day      (day) <U5 '11-01' '11-02' '11-03' ... '03-29' '03-30' '03-31'

In [18]:
##choose zonal mean meridional ehf at 60N
zm_ehf60 = zm_ehf.loc[dict(lat=60)]

In [19]:
zm_ehf60.shape

(44, 152)

In [24]:
pickle.dump(zm_ehf.values, open("1959ZMeddyheatflux.p", 'wb'))